In [6]:
!pip install ray==2.5.1 tensorflow pyarrow tblib

In [7]:
import argparse
from filelock import FileLock
import json
import os

import numpy as np
from ray.air.result import Result
import tensorflow as tf

from ray.train.tensorflow import TensorflowTrainer
from ray.air.integrations.keras import ReportCheckpointCallback
from ray.air.config import ScalingConfig


def mnist_dataset(batch_size: int) -> tf.data.Dataset:
    with FileLock(os.path.expanduser("~/.mnist_lock")):
        (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
    # The `x` arrays are in uint8 and have values in the [0, 255] range.
    # You need to convert them to float32 with values in the [0, 1] range.
    x_train = x_train / np.float32(255)
    y_train = y_train.astype(np.int64)
    train_dataset = (
        tf.data.Dataset.from_tensor_slices((x_train, y_train))
        .shuffle(60000)
        .repeat()
        .batch(batch_size)
    )
    return train_dataset


def build_cnn_model() -> tf.keras.Model:
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=(28, 28)),
            tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(32, 3, activation="relu"),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(10),
        ]
    )
    return model


def train_func(config: dict):
    per_worker_batch_size = config.get("batch_size", 64)
    epochs = config.get("epochs", 3)
    steps_per_epoch = config.get("steps_per_epoch", 70)

    tf_config = json.loads(os.environ["TF_CONFIG"])
    num_workers = len(tf_config["cluster"]["worker"])

    strategy = tf.distribute.MultiWorkerMirroredStrategy()

    global_batch_size = per_worker_batch_size * num_workers
    multi_worker_dataset = mnist_dataset(global_batch_size)

    with strategy.scope():
        # Model building/compiling need to be within `strategy.scope()`.
        multi_worker_model = build_cnn_model()
        learning_rate = config.get("lr", 0.001)
        multi_worker_model.compile(
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            metrics=["accuracy"],
        )

    history = multi_worker_model.fit(
        multi_worker_dataset,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        callbacks=[ReportCheckpointCallback()],
    )
    results = history.history
    return results


def train_tensorflow_mnist(
    num_workers: int = 2, use_gpu: bool = False, epochs: int = 4
) -> Result:
    config = {"lr": 1e-3, "batch_size": 64, "epochs": epochs}
    trainer = TensorflowTrainer(
        train_loop_per_worker=train_func,
        train_loop_config=config,
        scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
    )
    results = trainer.fit()
    return results

In [8]:
import ray

In [9]:
ray.init(address="ray://example-cluster-head-svc:10001")

RuntimeError: Ray Client is already connected. Maybe you called ray.init("ray://<address>") twice by accident?

In [ ]:
train_tensorflow_mnist(num_workers=2, use_gpu=True, epochs=100)

(TunerInternal pid=452) 2023-06-30 04:56:57.633525: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
(TunerInternal pid=452) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(TunerInternal pid=452) 2023-06-30 04:56:57.836576: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(TunerInternal pid=452) 2023-06-30 04:56:59.050186: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRA

(pid=665) 2023-06-30 04:57:08.195744: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
(pid=665) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=665) 2023-06-30 04:57:08.457044: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=665) 2023-06-30 04:57:09.941149: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
(

(RayTrainWorker pid=934) Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
11490434/11490434 [==============================] - 1s 0us/step


(RayTrainWorker pid=879) 2023-06-30 04:57:32.421919: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
(RayTrainWorker pid=879) op: "TensorSliceDataset"
(RayTrainWorker pid=879) input: "Placeholder/_0"
(RayTrainWorker pid=879) input: "Placeholder/_1"
(RayTrainWorker pid=879) attr {
(RayTrainWorker pid=879)   key: "Toutput_types"
(RayTrainWorker pid=879)   value {
(RayTrainWorker pid=879)     list {
(RayTrainWorker pid=879)       type: DT_FLOAT
(RayTrainWorker pid=879)       type: DT_INT64
(RayTrainWorker pid=879)     }
(RayTrainWorker pid=879)   }
(RayTrainWorker pid=879) }
(RayTrainWorker pid=879) attr {
(RayTrainWorker pid=879)   key: "_cardinality"
(RayTrainWorker pid=879)   value {
(RayTrainWorker pid=879)     i: 60000
(RayTrainWorker pid=879)   }
(RayTrainWorker pid=879) }
(Ra

(RayTrainWorker pid=879) Epoch 1/100
(RayTrainWorker pid=934) Epoch 1/100
70/70 [==============================] - ETA: 0s - loss: 4.5444 - accuracy: 0.4623
(TunerInternal pid=452) Result for TensorflowTrainer_38bb7_00000:
(TunerInternal pid=452)   accuracy: 0.23113839328289032
(TunerInternal pid=452)   date: 2023-06-30_04-58-00
(TunerInternal pid=452)   done: false
(TunerInternal pid=452)   hostname: example-cluster-head-vxm8k
(TunerInternal pid=452)   iterations_since_restore: 1
(TunerInternal pid=452)   loss: 2.2722084522247314
(TunerInternal pid=452)   node_ip: 10.42.0.75
(TunerInternal pid=452)   pid: 665
(TunerInternal pid=452)   should_checkpoint: true
(TunerInternal pid=452)   time_since_restore: 48.73028326034546
(TunerInternal pid=452)   time_this_iter_s: 48.73028326034546
(TunerInternal pid=452)   time_total_s: 48.73028326034546
(TunerInternal pid=452)   timestamp: 1688126280
(TunerInternal pid=452)   training_iteration: 1
(TunerInternal pid=452)   trial_id: 38bb7_00000
(Tun